# 108 LangGraph: Your First LLM Integration (Simple Bot)

**Workshop**: LangGraph 108
**Duration**: ~30 minutes
**Difficulty**: Intermediate

## Learning Objectives

By completing this notebook, you will:
- Integrate an LLM (Large Language Model) into your LangGraph workflows
- Define state structures for handling AI messages
- Initialize and invoke Anthropic's Claude models using LangChain
- Build a simple AI bot for PAN-OS firewall queries
- Understand the difference between a simple bot and a true conversational agent
- Discover the limitations of stateless AI interactions (no memory)

## Prerequisites

- **Knowledge**: Completed notebooks 101-107 (Type Annotations, Core Concepts, Graph Patterns)
- **Setup**: Anthropic API key
- **Cost**: Very minimal - Claude Haiku costs pennies per 1000 tokens

**Important**: This notebook introduces LLM integration after mastering graph patterns in notebooks 103-107. If you haven't completed those notebooks, please go back and work through the graph pattern foundations first.

## What You'll Build

In this notebook, we'll build a **Simple PAN-OS AI Bot** that can answer questions about firewall operations, upgrades, and configurations. However, it won't remember previous messages in the conversation - and that's intentional! This limitation will motivate the next notebook where we add memory.

**Graph Structure:**
```
START → process_query → END
```

Simple, but powerful - this is where AI meets LangGraph!

## Table of Contents

1. [Introduction](#1-introduction)
2. [Setup and Imports](#2-setup-and-imports)
3. [Understanding Human Messages](#3-understanding-human-messages)
4. [Defining the Agent State](#4-defining-the-agent-state)
5. [Initializing the LLM](#5-initializing-the-llm)
6. [Building the Agent Graph](#6-building-the-agent-graph)
7. [Testing the Simple Bot](#7-testing-the-simple-bot)
8. [The Memory Problem](#8-the-memory-problem)
9. [Summary](#9-summary)

---

## 1. Introduction

Welcome to your **first LLM-integrated LangGraph agent**! This is an exciting milestone in your workshop journey.

### What We're Building

In notebooks 103-107, we built LangGraph workflows without any AI. We learned:
- How to define state structures (103-104)
- How to create nodes and connect them (103)
- Sequential workflows, conditional routing, and loops (105-107)

But we haven't actually integrated an **LLM (Large Language Model)** yet. That changes today!

### The Simple Bot

We're building what I call a **Simple Bot** - a basic LLM wrapper that can:
- Accept questions about PAN-OS firewalls
- Send those questions to an AI model (Claude)
- Return intelligent responses

### Why "Simple" Bot?

This bot has a critical limitation: **it has no memory**. Each question is treated independently, with no context from previous messages. 

Imagine asking:
1. "My firewall is fw-prod-01"
2. "What's the hostname of my firewall?"

The bot won't remember you just told it! This limitation is intentional - it'll motivate the next notebook where we add conversation memory.

### Real-World Use Case

Even without memory, a simple bot is useful for:
- One-off firewall configuration queries
- Upgrade path recommendations
- Quick command syntax lookups
- Policy best practices

Let's get started!

---

## 2. Setup and Imports

### Important: LangChain + LangGraph Together

You might be wondering: "Wait, I thought we were learning LangGraph, why are we importing LangChain libraries?"

Great question! Here's the key insight:

**LangGraph is built on top of LangChain**. LangChain provides sophisticated, robust libraries for working with LLMs, and LangGraph leverages these strengths. Think of it as:
- **LangChain**: Tools for working with LLMs (models, messages, prompts)
- **LangGraph**: Framework for orchestrating LLMs into multi-step workflows

They work together beautifully!

### About API Keys

We need an API key because we're making calls to an external LLM service (Anthropic). If you were using a local LLM (like through Ollama), you wouldn't need an API key.

We'll store the API key in a `.env` file for security - never hardcode API keys in your notebooks!

### Cost Note

Don't worry about cost! Claude Haiku is extremely cheap:
- Input: ~$0.25 per 1 million tokens
- Output: ~$1.25 per 1 million tokens

For this workshop, you'll spend pennies at most.

In [ ]:
# Core typing imports
from typing import TypedDict, List

# LangChain message types - THIS IS NEW!
from langchain_core.messages import HumanMessage, AIMessage

# LangChain LLM integration - THIS IS NEW!
from langchain_anthropic import ChatAnthropic

# LangGraph core (familiar from notebooks 103-107)
from langgraph.graph import StateGraph, START, END

# Visualization
from IPython.display import Image, display

# Environment variable loading for API keys
from dotenv import load_dotenv

print("✅ All imports successful!")
print("\nWhat's new in this notebook:")
print("  - HumanMessage: Represents messages from users to the AI")
print("  - AIMessage: Represents responses from the AI to users")
print("  - ChatAnthropic: Interface to Anthropic's Claude models")
print("  - load_dotenv: Securely loads API keys from .env file")
print("\n💡 LangChain + LangGraph work together!")
print("   LangChain: LLM tools (models, messages)")
print("   LangGraph: Workflow orchestration (graphs, nodes)")

### 2.1 Load Environment Variables

Before we can use Anthropic's API, we need to load our API key from the `.env` file.

**Creating a .env file:**

If you don't have one yet, create a file named `.env` in your project root:

```bash
ANTHROPIC_API_KEY=sk-ant-your-api-key-here
```

**Security Note:** Never commit `.env` files to git! Add `.env` to your `.gitignore`.

In [ ]:
# Load environment variables from .env file
load_dotenv()

print("✅ Environment variables loaded!")
print("\n💡 The ANTHROPIC_API_KEY is now available for use")
print("   (but we won't print it for security reasons!)")

---

## 3. Understanding Human Messages

### What is a HumanMessage?

A `HumanMessage` is a structured way to represent a message from a human (you!) to an AI model. It's part of LangChain's message system.

### Why Not Just Use Strings?

You might think: "Why not just pass strings to the LLM?" 

LangChain uses typed message objects because:
1. **Type Safety**: Clear distinction between human messages, AI messages, system messages
2. **Metadata**: Messages can carry additional context (timestamps, IDs, etc.)
3. **Conversation History**: Easy to track who said what in multi-turn conversations
4. **Consistency**: Works across different LLM providers

### Message Types in LangChain

There are several message types:
- **HumanMessage**: From the user to the AI
- **AIMessage**: From the AI to the user
- **SystemMessage**: Instructions for the AI (like "You are a PAN-OS expert")
- **ToolMessage**: Results from tool/function calls

For now, we'll focus on `HumanMessage`.

Let's see how to create one!

In [ ]:
# Creating a simple HumanMessage
message = HumanMessage(content="What is the recommended upgrade path from PAN-OS 10.1 to 10.2?")

print("HumanMessage created:")
print(f"  Type: {type(message).__name__}")
print(f"  Content: {message.content}")
print(f"\nFull message object:")
print(message)

print("\n💡 HumanMessage wraps the content with metadata!")
print("   This tells the LLM: 'This message is from a human'")

### 3.1 Preview: SystemMessage (Coming in Notebook 109)

We've seen **HumanMessage** (user → AI) and **AIMessage** (AI → user). There's one more important message type: **SystemMessage**.

**What is SystemMessage?**

SystemMessage provides **instructions and context** to the AI before the conversation starts. Think of it as setting the AI's role, expertise, and personality.

**Example SystemMessage:**
```python
from langchain_core.messages import SystemMessage

system_msg = SystemMessage(content="""You are an expert PAN-OS network engineer 
specializing in Strata Cloud Manager automation. You provide clear, accurate 
technical guidance on firewall configuration, pan-scm-sdk usage, and security 
best practices. Always include working code examples when relevant.""")
```

**Why use SystemMessage?**
- **Set expertise**: Make the bot a PAN-OS expert, not generic assistant
- **Define personality**: Concise technical vs detailed explanatory
- **Establish context**: SCM focus, SDK patterns, security emphasis  
- **Guide responses**: Code examples, best practices, warnings

**Message order in conversation:**
```python
messages = [
    SystemMessage(content="You are a PAN-OS expert..."),  # First: Set context
    HumanMessage(content="How do I create a security rule?"),  # User asks
    AIMessage(content="Here's how to create a security rule..."),  # AI responds
    HumanMessage(content="What about NAT rules?"),  # Follow-up
    # ... conversation continues
]
```

**Preview for Notebook 109:**

In the next notebook, we'll use SystemMessage to:
- Give our bot PAN-OS/SCM expertise
- Improve response quality and relevance
- Maintain consistent technical tone
- Combine with conversation memory for powerful agents

For now, we're keeping it simple - just HumanMessage and AIMessage!

**Complete message type system:**
- **SystemMessage**: Instructions/context for the AI (we'll use in 109)
- **HumanMessage**: User input (we're using now)
- **AIMessage**: AI responses (we're using now)
- **ToolMessage**: Tool/function results (we'll use in 110)

Let's continue building our simple bot!

---

## 4. Defining the Agent State

Just like in notebook 103, we need to define a state structure for our graph. This time, our state will hold **messages**.

### State Structure

Our state will have a single field:
- `messages`: A list of HumanMessage objects

### Why List[HumanMessage]?

We use a list because:
1. Users might send multiple messages
2. We need to track message history (even though our simple bot won't use it)
3. It's the foundation for conversation memory (coming in the next notebook)

Let's define it!

In [ ]:
class AgentState(TypedDict):
    """State for our simple PAN-OS AI bot."""
    messages: List[HumanMessage]  # Only human messages - we don't save AI responses yet!

print("✅ AgentState defined!")
print("\nState structure:")
print("  - messages: List[HumanMessage]")
print("\n💡 This state will hold all the messages users send to our bot")
print("\n⚠️  Notice: We only track HumanMessage, not AIMessage!")
print("   This is intentional - our simple bot doesn't save AI responses.")

### 4.1 Why Only HumanMessage?

You might notice our state only tracks `HumanMessage` objects, not `AIMessage` responses.

**This is intentional!** Our simple bot:
- ✅ Receives user messages
- ✅ Sends them to the LLM
- ✅ Prints AI responses
- ❌ **Doesn't save AI responses to state**

This means **no conversation memory**. Each query is independent.

**In Notebook 109, we'll upgrade to:**
```python
from typing import Union

class AgentState(TypedDict):
    messages: List[Union[HumanMessage, AIMessage]]  # Both types!
```

This will allow us to store the full conversation history (human messages AND AI responses), enabling memory.

**For now, keep it simple** - just human messages, no AI response storage.

---

## 5. Initializing the LLM

Now for the exciting part - initializing our Large Language Model!

### Choosing a Model

We'll use **Claude 3.5 Haiku** (Anthropic's fast, cost-effective model) for this workshop. You could also use:
- **Claude 3.5 Sonnet**: More capable, slightly more expensive
- **Claude 3 Opus**: Most capable, highest cost
- **GPT-4 (OpenAI)**: Alternative provider via `ChatOpenAI`
- **Ollama**: Local models (no API key needed)

### Why ChatAnthropic?

LangChain provides a unified interface through classes like:
- `ChatAnthropic`: For Anthropic's Claude models
- `ChatOpenAI`: For OpenAI's GPT models
- `ChatOllama`: For local Ollama models

They all work the same way, so switching providers is easy!

### Personal Note

I personally use `ChatAnthropic` because:
- Excellent performance and reasoning
- Very affordable (especially Haiku)
- Strong privacy and safety features
- Easy integration with LangChain

Let's initialize it!

In [ ]:
# Initialize the LLM - this is where the AI magic happens!
llm = ChatAnthropic(model="claude-3-5-haiku-20241022")

print("✅ LLM initialized!")
print("\nModel details:")
print("  - Provider: Anthropic")
print("  - Model: Claude 3.5 Haiku")
print("  - Cost: ~$0.25 per 1M input tokens, ~$1.25 per 1M output tokens")
print("\n💡 For more capable model, use 'claude-3-5-sonnet-20241022' instead!")
print("\n⚠️  Make sure your ANTHROPIC_API_KEY is set in .env file")

### 5.1 Quick LLM Test (Optional)

Let's do a quick test to make sure our LLM is working before we build the full graph.

**The invoke() Method:**

You'll notice LangChain loves the word `invoke()`. We've used it to run graphs, and now we use it to call the LLM:
- `graph.invoke()`: Run the graph
- `llm.invoke()`: Call the LLM

Consistent naming makes it easy to remember!

In [ ]:
# Quick test: Ask the LLM a simple PAN-OS question
test_message = HumanMessage(content="What is PAN-OS?")
test_response = llm.invoke([test_message])

print("Test Question: What is PAN-OS?")
print("\nAI Response:")
print(test_response.content)
print("\n✅ LLM is working! Ready to build our agent.")

### 5.2 Understanding AIMessage Responses

When you call `llm.invoke()`, the response is not a plain string - it's an **AIMessage** object!

**Why AIMessage?**
- Structured representation of AI responses
- Contains metadata (model used, tokens, timing)
- Matches the HumanMessage pattern for consistency
- Enables conversation history tracking (coming in notebook 109)

Just like we use `HumanMessage` to represent messages FROM users TO the AI, we use `AIMessage` to represent messages FROM the AI TO users.

**Message Flow:**
```
User → HumanMessage → LLM → AIMessage → Response
```

Let's examine the response object more closely!

In [ ]:
# Look at the test response from the previous cell
print("Response Object Analysis:")
print(f"  Type: {type(test_response).__name__}")
print(f"  Is AIMessage?: {isinstance(test_response, AIMessage)}")

print("\n" + "=" * 70)
print("AIMessage Structure:")
print("=" * 70)
print(f"\n📝 Content (what the AI said):")
print(f"   {test_response.content[:150]}...")  # First 150 chars

print(f"\n🏷️  Message Type:")
print(f"   {test_response.type}")

print(f"\n📊 Response Metadata:")
for key, value in test_response.response_metadata.items():
    print(f"   {key}: {value}")

print("\n" + "=" * 70)
print("💡 Key Insight:")
print("=" * 70)
print("   HumanMessage: User → AI")
print("   AIMessage:    AI → User")
print("\n   Both are message objects with .content and metadata!")
print("   This symmetry makes conversation tracking easy!")

---

## 6. Building the Agent Graph

Now let's build our simple bot using everything we've learned!

### The Process Node

We'll create a node that:
1. Takes the user's messages from state
2. Sends them to the LLM using `invoke()`
3. Gets the AI's response
4. Prints the response (for now)

### Graph Structure

Remember from notebook 103? Our graph structure is:
```
START → process_query → END
```

Simple, but now with AI power!

Let's build it!

### 6.1 Define the Process Node

In [ ]:
def process_query(state: AgentState) -> AgentState:
    """
    Node: Process user query by sending it to the LLM.
    
    This is where the AI magic happens!
    
    Note: We print the response but DON'T save it to state.
    This means no memory between queries (fixed in notebook 109).
    """
    # Get the messages from state
    messages = state["messages"]
    
    # Send messages to LLM and get response (returns AIMessage object)
    # The llm.invoke() method calls Anthropic's API
    response = llm.invoke(messages)
    
    # Print the AI's response content
    print("\n🤖 AI Response:")
    print(response.content)
    print()
    
    # ⚠️ IMPORTANT: We return unchanged state (no memory!)
    # We're NOT doing: state["messages"].append(response)
    # This means the bot won't remember this conversation
    return state

print("✅ process_query node defined!")
print("\nWhat this node does:")
print("  1. Reads messages from state")
print("  2. Invokes the LLM with those messages")
print("  3. Gets AIMessage response from Anthropic's servers")
print("  4. Prints the response content")
print("  5. Returns UNCHANGED state (no memory!)")
print("\n💡 The invoke() method handles all the API communication!")
print("\n⚠️  The missing step: We never save the AI response!")
print("   This is why the bot has no memory (fixed in notebook 109)")

### 6.1.1 Production Pattern: Error Handling for LLM Calls

Before we build our graph, let's learn about **production error handling** for LLM calls.

### Why Error Handling Matters

When calling external APIs like Anthropic, things can go wrong:
- **Network failures**: Connection drops, timeouts
- **API errors**: Rate limits, invalid API keys, service outages  
- **Invalid responses**: Malformed data, unexpected formats

In production SCM automation, you need **resilient bots** that handle these gracefully!

### Common LLM API Errors

**1. Invalid API Key**
```
Error: 401 Unauthorized - Invalid API key
```
Fix: Check your `.env` file and ANTHROPIC_API_KEY

**2. Rate Limiting**
```
Error: 429 Too Many Requests - Rate limit exceeded
```
Fix: Implement retry logic with exponential backoff (like notebook 107!)

**3. Network Failures**
```
Error: ConnectionError - Failed to reach API
```
Fix: Check internet connection, retry after delay

**4. Timeout**
```
Error: TimeoutError - Request took too long
```
Fix: Increase timeout or retry

### Error Handling Pattern

Here's how to wrap `llm.invoke()` with production error handling:

In [ ]:
def process_query_with_error_handling(state: AgentState) -> AgentState:
    """
    Node: Process user query with production error handling.
    
    Handles common LLM API errors:
    - ConnectionError: Network failures
    - Timeout errors: Request took too long
    - API errors: Rate limits, invalid keys, service outages
    - Generic exceptions: Catch-all for unexpected issues
    """
    messages = state["messages"]
    
    try:
        # Call the LLM (this might fail!)
        response = llm.invoke(messages)
        
        # Success! Print the response
        print("\n🤖 AI Response:")
        print(response.content)
        print()
        
    except ConnectionError as e:
        # Network failure - can't reach Anthropic's API
        print("\n❌ Connection Error:")
        print(f"   {str(e)}")
        print("   → Check your internet connection")
        print("   → Try again in a moment")
        
    except TimeoutError as e:
        # Request took too long
        print("\n❌ Timeout Error:")
        print(f"   {str(e)}")
        print("   → The LLM didn't respond in time")
        print("   → Try a shorter query or retry")
        
    except Exception as e:
        # Catch-all for other errors (invalid API key, rate limits, etc.)
        error_message = str(e)
        
        # Check for common error patterns
        if "401" in error_message or "unauthorized" in error_message.lower():
            print("\n❌ Authentication Error:")
            print("   Invalid API key")
            print("   → Check your .env file")
            print("   → Verify ANTHROPIC_API_KEY is correct")
            
        elif "429" in error_message or "rate limit" in error_message.lower():
            print("\n❌ Rate Limit Error:")
            print("   Too many requests too quickly")
            print("   → Wait a moment and try again")
            print("   → Consider implementing retry logic (notebook 107!)")
            
        else:
            # Unknown error
            print("\n❌ Unexpected Error:")
            print(f"   {error_message}")
            print("   → Check the error message above")
            print("   → Verify your API key and network connection")
    
    # Always return state (even if error occurred)
    return state

print("✅ process_query_with_error_handling node defined!")
print("\nWhat this node does:")
print("  1. Wraps llm.invoke() in try-catch block")
print("  2. Handles ConnectionError (network failures)")
print("  3. Handles TimeoutError (slow responses)")
print("  4. Handles API errors (401 auth, 429 rate limits)")
print("  5. Provides helpful error messages for debugging")
print("  6. Always returns state (graceful degradation)")
print("\n💡 This is production-ready error handling!")
print("\n⚠️  For this notebook, we'll use the simpler process_query")
print("   But in production SCM automation, use this pattern!")

### 6.2 Build and Compile the Graph

This should look familiar from notebooks 103-107!

In [ ]:
# Create the graph
graph = StateGraph(AgentState)

# Add the process node
graph.add_node("process_query", process_query)

# Set entry and exit points
graph.set_entry_point("process_query")
graph.set_finish_point("process_query")

# Compile the graph
agent = graph.compile()

print("✅ Graph built and compiled!")
print("\nGraph structure:")
print("  START → process_query → END")
print("\n💡 Same structure as notebook 103, but now with LLM power!")

### 6.3 Visualize the Graph

In [ ]:
# Visualize the graph
display(Image(agent.get_graph().draw_mermaid_png()))

print("\n💡 Notice: It looks like our first graph from notebook 103!")
print("   But now the 'process_query' node contains AI magic!")

---

## 7. Testing the Simple Bot

Time to test our AI agent! We'll start with a single question, then try multiple questions.

### How It Works

1. User enters a question about PAN-OS
2. We wrap it in a `HumanMessage`
3. We invoke the graph with the message
4. The LLM processes it and responds

Let's try it!

### 7.1 Single Query Test

### 7.2 Multiple Queries with While Loop

Let's make it more interactive! We'll use a while loop so you can ask multiple questions.

**How to exit:** Type `exit` or `quit` when done.

### 7.3 Example Queries to Try

Here are some PAN-OS questions to try with your bot:

**Upgrade Questions:**
- "What is the recommended upgrade path from PAN-OS 10.1 to 10.2?"
- "How do I check if my firewall is ready for an upgrade?"
- "What are the steps for upgrading an HA pair?"

**Configuration Questions:**
- "How do I create a security policy rule?"
- "What's the difference between a security zone and a zone protection profile?"
- "How do I configure NAT on a PAN-OS firewall?"

**Troubleshooting Questions:**
- "What does error code 'session discard' mean?"
- "How do I check CPU and memory usage on PAN-OS?"
- "Why is my firewall dropping traffic?"

**Strata Cloud Manager (SCM) Specific Questions:**
- "How do I create an address object in Strata Cloud Manager using pan-scm-sdk?"
- "What's the difference between folder, snippet, and device containers in SCM?"
- "How do I list all security policies in a specific folder using the SDK?"
- "What are the required fields for creating a security rule in SCM?"
- "How do I handle pagination when listing address objects in SCM?"
- "What's the best practice for organizing SCM configuration across folders?"

**Try asking follow-up questions and see what happens!**

💡 **Hint**: Pay special attention to whether the bot remembers context from your previous questions. You'll notice it doesn't - that's the memory problem we'll solve in the next notebook!

### 7.4 Bot Providing SCM SDK Code Examples

One powerful use case for our bot: **generating pan-scm-sdk code snippets**!

Even without memory, the bot can provide working SDK examples based on your questions. Let's try a few SCM-specific queries and see the bot generate actual code you can use.

**How this helps:**
- Get instant SDK examples without searching documentation
- Learn correct pan-scm-sdk patterns
- Copy working code directly into your automation scripts
- Understand required parameters and data structures

Let's test it!

In [ ]:
print("=" * 70)
print("SCM SDK CODE GENERATION DEMO")
print("=" * 70)

# Example 1: Creating an address object
print("\n📝 Example 1: Creating an Address Object")
print("-" * 70)
query1 = HumanMessage(content="Show me how to create an address object using pan-scm-sdk with the following: name 'web-server-01', IP address '10.1.1.100/32', folder 'Texas', and description 'Production web server'")
agent.invoke({"messages": [query1]})

# Example 2: Listing security rules with pagination
print("\n📝 Example 2: Listing Security Rules with Pagination")
print("-" * 70)
query2 = HumanMessage(content="Show me pan-scm-sdk code to list all security rules in the 'Texas' folder with proper pagination handling")
agent.invoke({"messages": [query2]})

# Example 3: Error handling for SCM operations
print("\n📝 Example 3: SCM Exception Handling")
print("-" * 70)
query3 = HumanMessage(content="Show me how to handle exceptions when creating an address object in pan-scm-sdk, including InvalidObjectError and NameNotUniqueError")
agent.invoke({"messages": [query3]})

print("\n" + "=" * 70)
print("💡 KEY INSIGHT: LLM-Generated SDK Code")
print("=" * 70)
print("\nThe bot provided working pan-scm-sdk code snippets!")
print("\n✅ Benefits:")
print("   - Instant SDK examples without documentation search")
print("   - Correct import statements and client initialization")
print("   - Proper data models and required fields")
print("   - Exception handling patterns")
print("   - Ready to copy-paste into your automation scripts")
print("\n⚠️  Remember: Always review and test generated code!")
print("   - Verify folder names match your environment")
print("   - Check API credentials are correctly loaded")
print("   - Test in development before production")
print("\n🔗 Reference: See docs/examples/ for more SDK patterns")

---

## 8. The Memory Problem

Now let's discover the critical limitation of our simple bot.

### The Experiment

Try this sequence of questions:

1. **First message:** "My firewall hostname is fw-prod-01"
2. **Second message:** "What is my firewall's hostname?"

### What Happens?

The bot will respond with something like: "I don't have information about your firewall's hostname."

**Why?** Because our bot has **no memory**. Each question is processed independently with no context from previous messages.

### The Code Explanation

Look at our `process_query` node:

```python
def process_query(state: AgentState) -> AgentState:
    messages = state["messages"]
    response = llm.invoke(messages)
    # We print the response but DON'T save it to state!
    return state  # State unchanged - no memory of the conversation
```

**The problem:** We never update the state with:
- The AI's response
- The conversation history
- Any context from previous messages

### Why This Matters

For real-world PAN-OS automation, you need:
- Multi-turn troubleshooting conversations
- Context-aware recommendations
- Ability to reference previous messages

### The Solution (Coming Next!)

In the next notebook, we'll learn about:
- **Conversation memory**: Storing message history
- **State reducers**: The `Annotated[list, add_messages]` pattern
- **True conversational agents**: Bots that remember context

But for now, you've successfully integrated an LLM into LangGraph - that's a huge milestone!

### 8.1 Demonstrate the Memory Problem

Let's run a concrete example showing the limitation:

In [ ]:
print("=" * 70)
print("DEMONSTRATING THE MEMORY PROBLEM")
print("=" * 70)

# First message: Tell the bot our firewall name
print("\n1️⃣ First Query: 'My firewall hostname is fw-prod-01'")
message1 = HumanMessage(content="My firewall hostname is fw-prod-01")
agent.invoke({"messages": [message1]})

# Second message: Ask what the firewall name is
print("\n2️⃣ Second Query: 'What is my firewall hostname?'")
message2 = HumanMessage(content="What is my firewall hostname?")
agent.invoke({"messages": [message2]})

print("\n" + "=" * 70)
print("❌ PROBLEM: The bot doesn't remember we told it 'fw-prod-01'!")
print("=" * 70)
print("\nWhy? Because we never stored the conversation history in state.")
print("Each query is independent - no memory between messages.")
print("\n💡 Next notebook: We'll fix this with conversation memory!")

---

## 9. Summary

🎉 **Congratulations!** You've built your first AI-powered LangGraph agent!

### What We Accomplished

1. **Integrated an LLM** - Connected Anthropic's Claude to our graph
2. **Used HumanMessage** - Learned how LangChain structures messages
3. **Built a Simple Bot** - Created a single-node graph with AI capabilities
4. **Tested with PAN-OS Queries** - Asked real firewall questions
5. **Discovered the Memory Problem** - Learned why stateless bots are limited

### Key Concepts Learned

✅ **LangChain + LangGraph Work Together**
   - LangChain: LLM tools (ChatAnthropic, messages)
   - LangGraph: Workflow orchestration (graphs, nodes)

✅ **HumanMessage Objects**
   - Structured way to represent user messages
   - Better than plain strings for conversation management

✅ **The invoke() Pattern**
   - `llm.invoke()`: Call the LLM
   - `agent.invoke()`: Run the graph
   - Consistent naming across LangChain/LangGraph

✅ **State Management**
   - Defined `AgentState` with `List[HumanMessage]`
   - State flows through the graph like before

✅ **The Memory Problem**
   - Simple bots don't remember previous messages
   - Each query is independent
   - This motivates the next notebook!

### Why This Matters

Even though our bot lacks memory, you've learned the fundamental pattern for integrating LLMs into LangGraph:

```python
# 1. Define state with messages
class AgentState(TypedDict):
    messages: List[HumanMessage]

# 2. Initialize LLM
llm = ChatAnthropic(model="claude-3-5-haiku-20241022")

# 3. Create node that invokes LLM
def process(state):
    response = llm.invoke(state["messages"])
    return state

# 4. Build graph as usual
graph.add_node("process", process)
```

This pattern is the foundation for ALL LLM agents in LangGraph!

### Real-World Use Cases (Without Memory)

Even simple bots are useful for:
- **One-off queries**: "What's the CLI command for X?"
- **Quick lookups**: "What does error code Y mean?"
- **Documentation search**: "How do I configure Z?"
- **Best practices**: "What's the recommended approach for X?"

### What's Next?

In the next notebook (**109: Conversational Memory**), we'll learn:
- How to add **conversation memory**
- The `Union[HumanMessage, AIMessage]` **pattern**
- **Multi-turn conversations** with context
- **AIMessage** for storing LLM responses
- Building **true conversational agents**

### Code Summary

Our complete simple bot in ~25 lines:

```python
# State
class AgentState(TypedDict):
    messages: List[HumanMessage]

# LLM
llm = ChatAnthropic(model="claude-3-5-haiku-20241022")

# Node
def process_query(state: AgentState) -> AgentState:
    response = llm.invoke(state["messages"])
    print(response.content)
    return state

# Graph
graph = StateGraph(AgentState)
graph.add_node("process_query", process_query)
graph.set_entry_point("process_query")
graph.set_finish_point("process_query")
agent = graph.compile()

# Run
agent.invoke({"messages": [HumanMessage(content="Hi!")]})
```

Simple, clean, and powerful!

### Final Thoughts

You've crossed a major milestone - you can now integrate AI into your workflows! The journey from here gets even more exciting as we add memory, tools, and sophisticated agent behaviors.

**Great work!** 🚀

---

### Next Workshop Preview

In **Notebook 109: Conversational Memory**, we'll solve the memory problem:

```
You: My firewall is fw-prod-01
AI: Got it! I'll remember that your firewall is fw-prod-01.

You: What's my firewall's hostname?
AI: Your firewall's hostname is fw-prod-01.
```

See you there! 💡